In [27]:
# GENERAL SETTINGS --------------------------------------------------------------------------------------------------------------------

# Packages
## Spark packages
import pyspark
from pyspark.sql import SparkSession
## Manipulation packages
from pyspark.sql.functions import col, size, split, max
import pyspark.pandas as ps
## General packages
import os

# Paths
## Main directory
module_path = os.path.abspath(os.path.join('..'))
## raw directory
raw_data_path = module_path + "/data/raw/"

# Create spark session
spark = (
    SparkSession.builder.appName("challenge1_module2")
    .getOrCreate()
)

# Show only 20 rows with Pandas API
ps.options.display.max_rows = 20

print('SUCCESS - GENERAL SETTINGS\n')

SUCCESS - GENERAL SETTINGS



In [28]:
# EXTRACTION --------------------------------------------------------------------------------------------------------------------
## Reading data from raw layer
raw_ratings = spark.read.csv(raw_data_path + 'title_ratings.tsv', header=True, inferSchema=True, sep='\t')
pd_raw_ratings = raw_ratings.pandas_api()
raw_basics = spark.read.csv(raw_data_path + 'title_basics.tsv', header=True, inferSchema=True, sep='\t')
pd_raw_basics = raw_basics.pandas_api()
print('SUCCESS - EXTRACTION\n')

SUCCESS - EXTRACTION



In [29]:
# CONSUMPTION --------------------------------------------------------------------------------------------------------------------
## Answers

### 4. Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?
q4 = raw_basics.filter((raw_basics.startYear == 2015) & ((raw_basics.titleType == 'movie') | (raw_basics.titleType == 'tvMovie'))).count()
print('\n4 answer =', q4, '\n')




4 answer = 19987 



In [30]:
### 5. Qual o gênero de títulos mais frequente?
#### Max of columns before the split
n_genres_max_per_row = raw_basics.withColumn('n', size(split(col("genres"), r","))).select(max('n')).collect()[0][0]
#### Split into 'n' columns
df_genres = (raw_basics.withColumn('temp', split('genres', '\\,')).select(*(col('temp').getItem(i).alias(f'genre_{i}') for i in range(n_genres_max_per_row))))
#### Stack the columns in only one column
genre_0 = df_genres.filter(col('genre_0').isNotNull()).select('genre_0')
genre_1 = df_genres.filter(col('genre_1').isNotNull()).select('genre_1')
genre_2 = df_genres.filter(col('genre_2').isNotNull()).select('genre_2')
genre_0_1 = genre_0.union(genre_1)
genre_0_1_2 = genre_0_1.union(genre_2)
#### Get the more frequent value
q5 = genre_0_1_2.groupBy('genre_0').count().sort(col("count").desc()).select('genre_0').collect()[0][0]

print('\n5 answer =', q5, '\n')


5 answer = Drama 



In [31]:
### 6. Qual o gênero com a melhor nota média de títulos?
q6 = 'to do'
print('\n6 answer =', q6, '\n')




6 answer = to do 



In [32]:
### 7. Qual o vídeo game do gênero aventura mais bem avaliado em 2020?
q7 = 'to do'
print('\n7 answer =', q7, '\n')




7 answer = to do 



In [33]:
### 9. Quantos títulos de filmes diferentes existem? Use df_titles.select('primaryTitle').distinct().count().
q9 = 'to do'
print('\n9 answer =', q9, '\n')




9 answer = to do 



In [34]:
### 10. Qual a duração média dos filmes com conteúdo adulto? Use uma combinação de filter() e describe().
q10 = 'to do'
print('\n10 answer =', q10, '\n')




10 answer = to do 



In [35]:
### 11. Quantos filmes têm o título atual (“primary”) diferente do título original? Use uma combinação de filter, e count().
q11 = 'to do'
print('\n11 answer =', q11, '\n')




11 answer = to do 



In [36]:
### 12. Qual o filme que tem o nome mais longo?
q12 = 'to do'
print('\n12 answer =', q12, '\n')




12 answer = to do 



In [37]:
### 13. Qual filme tem a maior quantidade de votos? Dica: Use describe().
q13 = 'to do'
print('\n13 answer =', q13, '\n')



13 answer = to do 



In [38]:
### 15. Qual é a menor nota média de um filme? Use describe().
q15 = 'to do'
print('\n15 answer =', q15, '\n')


print('SUCCESS - CONSUMPTION\n')


15 answer = to do 

SUCCESS - CONSUMPTION

